In [3]:
# Install Panel
!pip install panel
!pip install altair
!pip install plotly

In [6]:
# Import Libraries
import json, os
from pathlib import Path
import altair as alt
import plotly.io as pio
import panel as pn

pn.extension('vega', 'plotly') # Initialize Panel Extensions


In [8]:
# Paths to exported JSONs

exports = Path('exports')

files = {
    'c1': exports/'c1.v1.json',
    'c2': exports/'c2.v1.json',
    'p3': exports/'p3.plotly.json',
    'p4': exports/'p4.plotly.json',
    'p5': exports/'p5.plotly.json'
}

In [9]:
# Define functions for loading  plots/charts

def load_altair(path: Path, height=320):
    if not path.exists(): return None
    spec = json.loads(path.read_text(encoding="utf-8"))
    # Make Altair responsive to container width
    spec["width"] = "container"
    chart = alt.Chart.from_dict(spec)
    return pn.pane.Vega(chart, height=height, sizing_mode="stretch_width")

def load_plotly(path: Path, height=320):
    if not path.exists(): return None
    fig = pio.read_json(path.as_posix())
    fig.update_layout(autosize=True, height=height, margin=dict(t=50, r=20, b=50, l=20))
    return pn.pane.Plotly(fig, config={"responsive": True})

In [11]:
# Build Panes

panes = {
    "Altair 1": load_altair(files["c1"]),
    "Altair 2": load_altair(files["c2"]),
    "Plotly 3": load_plotly(files["p3"]),
    "Plotly 4": load_plotly(files["p4"]),
    "Plotly 5": load_plotly(files["p5"]),
}

# filter out None
panes = {k:v for k,v in panes.items() if v is not None}

# ---- BootstrapTemplate (nice, responsive) ----
template = pn.template.BootstrapTemplate(title="Austin Housing — Dashboard")

# Optional: light/dark toggle in navbar
template.header.append(pn.Row(
    pn.pane.Markdown("**Altair + Plotly**"), sizing_mode="stretch_width"
))

# Lay out in a simple grid (2 per row). Adjust as you like.
cards = []
for name, pane in panes.items():
    cards.append(pn.Card(pane, header=name, sizing_mode="stretch_both"))

# Place cards in rows of two
rows = []
for i in range(0, len(cards), 2):
    rows.append(pn.Row(*cards[i:i+2], sizing_mode="stretch_width"))

template.main[:] = rows if rows else [pn.pane.Markdown("No charts found.")]

In [12]:
# ---- Export ONE standalone HTML ----
out = "dashboard.html"
template.save(out, resources="inline")
print(f"Saved {out}")

Saved dashboard.html
